In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import math
%matplotlib inline

In [2]:
# videoFile = "Dataset/Train Tom and Jerry.mp4"
# imageFolder = "Dataset/frameTrain"
# cap = cv2.VideoCapture(videoFile)
# frameRate = cap.get(5)
# count = 0
# while( cap.isOpened() ):
#     frameId = cap.get(1)
#     ret, frame = cap.read()
#     if(ret != True):
#         break
#     if(frameId % math.floor(frameRate) == 0 ):
#         filename = imageFolder + "/frame" + str(count) + ".jpg"
#         cv2.imwrite(filename,frame)
#         count = count + 1
        
# cap.release()
# print("Done!")


In [3]:
# videoFile = "Dataset/Test Tom and Jerry.mp4"
# imageFolder = "Dataset/frameTest"
# cap = cv2.VideoCapture(videoFile)
# frameRate = cap.get(5)
# count = 0
# while( cap.isOpened() ):
#     frameId = cap.get(1)
#     ret, frame = cap.read()
#     if(ret != True):
#         break
#     if(frameId % math.floor(frameRate) == 0 ):
#         filename = imageFolder + "/test" + str(count) + ".jpg"
#         cv2.imwrite(filename,frame)
#         count = count + 1
        
# cap.release()
# print("Done!")

In [4]:
train = pd.read_csv('dataset/train.csv')
test  = pd.read_csv('dataset/test.csv')

In [5]:
train.shape

(298, 2)

In [6]:
s = train['Emotion'].tolist()

In [7]:
# s

In [8]:
train['Emotion'].unique()

array(['happy', 'surprised', 'angry', 'Unknown', 'sad'], dtype=object)

In [9]:
from collections import Counter

print(Counter(s).keys()) # equals to list(set(words))
print(Counter(s).values()) # counts the elements' frequency
print(len(Counter(s).keys()))

dict_keys(['happy', 'surprised', 'angry', 'Unknown', 'sad'])
dict_values([37, 68, 37, 116, 40])
5


In [10]:
tg_dict = {"happy":0, "surprised": 1, "angry": 2,"Unknown":3, "sad":4}
def label_encode(x):
    return tg_dict[x]

In [11]:
train['Emotion'] = train['Emotion'].apply(label_encode)

frame_ID = train['Frame_ID'].tolist()
emotion = train['Emotion'].tolist()

In [33]:
# emotion

In [12]:
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, Activation, Flatten,Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
import os
import glob
import tensorflow as tf

In [13]:
features=[]
labels=[]
path = 'Dataset/frameTrain/'
for i in range(0,298):
  if os.path.isfile(path+str(frame_ID[i])):
    pic = image.load_img(path+str(frame_ID[i]), target_size=(224, 224))
    #print(path+str(images[i]))
    x = image.img_to_array(pic)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features.append(x)
    labels.append(emotion[i])
  else:
    print(path+str(frame_ID[i]), 'not present')

In [14]:
len(features)

298

In [15]:
npfeatures = np.array(features)
print(npfeatures.shape)
img_dt = np.rollaxis(npfeatures, 1, 0)
print(img_dt.shape)
X = img_dt[0]
print(X.shape)
labels = np.array(emotion)
Y = to_categorical(emotion,5)
print(Y.shape)
# print(Y)

(298, 1, 224, 224, 3)
(1, 298, 224, 224, 3)
(298, 224, 224, 3)
(298, 5)


In [16]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [34]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [35]:
IMAGE_SIZE=[224,224]
pretrained_model = MobileNet(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
pretrained_model.trainable = False

In [36]:
model = Sequential([
        pretrained_model,
        GlobalAveragePooling2D(),
        Dense(60, activation='relu'),
        Dense(30, activation='relu'),
        
        Dense(5, activation='softmax')
    ])
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

In [37]:
model.compile(optimizer = 'adamax', loss='categorical_crossentropy', metrics=[f1_m])    
model.fit(X, Y, batch_size=32, epochs=10, validation_split=.1,callbacks=[es])

Train on 268 samples, validate on 30 samples
Epoch 1/10
268/268 [==============================] - 87s 325ms/sample - loss: 1.5472 - f1_m: 0.0757 - val_loss: 1.0388 - val_f1_m: 0.6957
Epoch 2/10
268/268 [==============================] - 80s 300ms/sample - loss: 1.2344 - f1_m: 0.3492 - val_loss: 0.8792 - val_f1_m: 0.6809
Epoch 3/10
268/268 [==============================] - 78s 293ms/sample - loss: 1.0919 - f1_m: 0.4406 - val_loss: 0.8809 - val_f1_m: 0.6809
Epoch 4/10
268/268 [==============================] - 80s 297ms/sample - loss: 0.9840 - f1_m: 0.5303 - val_loss: 0.8914 - val_f1_m: 0.7083
Epoch 5/10
268/268 [==============================] - 78s 292ms/sample - loss: 0.8711 - f1_m: 0.6019 - val_loss: 0.9422 - val_f1_m: 0.6939
Epoch 6/10
268/268 [==============================] - 79s 294ms/sample - loss: 0.8449 - f1_m: 0.5866 - val_loss: 0.8701 - val_f1_m: 0.6939
Epoch 7/10
268/268 [==============================] - 79s 293ms/sample - loss: 0.7302 - f1_m: 0.6862 - val_loss: 1.0303 -

In [38]:
images_test = test['Frame_ID'].tolist()
test_features=[]
path_test = 'Dataset/frameTest/'

In [39]:
len(images_test)

186

In [40]:
for i in range(0,186):
  if os.path.isfile(path_test+str(images_test[i])):
    pic = image.load_img(path_test+str(images_test[i]), target_size=(224, 224))
    #print(path+str(images[i]))
    x = image.img_to_array(pic)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    test_features.append(x)
  else:
    print(path_test+str(images_test[i]), 'not present')

In [41]:
test_features = np.array(test_features)
print(test_features.shape)
test_features = np.rollaxis(test_features, 1, 0)
print(test_features.shape)
X_test = test_features[0]
print(X_test.shape)

(186, 1, 224, 224, 3)
(1, 186, 224, 224, 3)
(186, 224, 224, 3)


In [42]:
preds = model.predict(X_test)

In [43]:
predictions=[]
for i in preds:
    predictions.append(np.argmax(i))

In [44]:
len(predictions)

186

In [45]:
test['Emotion'] = predictions

In [46]:
gt_dict = dict((v,k) for k,v in tg_dict.items())

def inverse_encode(x):
    return gt_dict[x]

test['Emotion'] = test['Emotion'].apply(inverse_encode)

In [47]:
test.head(1)

,Frame_ID,Emotion
0,test0.jpg,angry


In [48]:
test.to_csv('Submission.csv',header=True,index = None)